# Imports

In [ ]:
import pandas as pd
import requests
import numpy as np
import pandas as pd
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
import os
import plotly.express as px
import time

# API Imports

### Looking at the census data one comes to a big assumption that the data is very clean. Aquiring the dataset via the URL thoughwas found to be difficult. Different sections of the census repository were stored by different means. While some were in folders. This dataset was in a dashboard sysytem that supported API queries. Not only did this make the data collection easier but it also allowed for a skill that isnt used often to be utilied. I normally use Pandas Read function for all of my queries.

In [ ]:
def fetch_median_income_by_county(api_key, years):
    all_years_data = []
# This for loop takes an API key and queries the US Census database for the required demographic data. The URLs are standardized so its possible to query then in a loop by year.

# Loop through each year and request data from the Census ACS 5-Year API
    for year in years:
        url = f'https://api.census.gov/data/{year}/acs/acs5'
        params = {
            'get': 'NAME,B19013_001E',
            'for': 'county:*',
            'key': api_key
        }
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
# Their were many features in this dataset but the following were the only ones required for the purpose of this query.

# Create DataFrame from API response
        df = pd.DataFrame(data[1:], columns=data[0])

# Rename columns for clarity
        df = df.rename(columns={
            'NAME': 'County',
            'B19013_001E': 'Median_Household_Income',
            'state': 'State_Code',
            'county': 'County_Code'
        })
# An additional column was added using the year of query to organize each dataset prior to concatination.
        df['Year'] = year
        all_years_data.append(df)
    df_all_years = pd.concat(all_years_data, ignore_index=True)
# Convert income to numeric, coercing non-numeric entries to NaN
    df_all_years['Median_Household_Income'] = pd.to_numeric(df_all_years['Median_Household_Income'], errors='coerce')
# FIPS were not specifically available, state and county codes were merged to create the required FIPS code. This will be required to join other datasets to this one.
    df_all_years['FIPS'] = df_all_years['State_Code'] + df_all_years['County_Code']
    all_years_data_income = df_all_years
    return all_years_data_income

In [ ]:
all_years_data_income = fetch_median_income_by_county('92d47be7e95136939cd750005e2649fd073627e6', ['2012','2013','2014','2015','2016','2017','2018', '2019', '2020', '2021', '2022', '2023'])

# Borrowed Nathan RedFin Import and cleaning

In [ ]:
df_housing_market = pd.read_csv('https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/county_market_tracker.tsv000.gz', sep='\t')
df_inflation = pd.read_csv('https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23ebf3fb&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1320&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=CPIAUCSL&scale=left&cosd=1947-01-01&coed=2025-04-01&line_color=%230073e6&link_values=false&line_style=solid&mark_type=none&mw=3&lw=3&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2025-06-06&revision_date=2025-06-06&nd=1947-01-01')

while True:
    try:
        df_county_FIPS = pd.read_csv('https://www2.census.gov/geo/docs/reference/codes2020/national_county2020.txt', sep='|')
        break
    except Exception as e:
        time.sleep(3)

In [ ]:
# Combine state abbv and county name to match format found in Redfin housing market data.
df_county_FIPS['county_and_state'] = [f"{row[1][4]}, {row[1][0]}" for row in df_county_FIPS.iterrows()]

# Concat state FIPS and county FIPS, add leading zeros up to 5 digits.
df_county_FIPS['county_and_state_FIPS'] = df_county_FIPS.STATEFP*1000+df_county_FIPS.COUNTYFP
df_county_FIPS['county_and_state_FIPS'] = df_county_FIPS['county_and_state_FIPS'].astype(str).str.zfill(5)


In [ ]:
df_housing_market['REGION_lower'] = df_housing_market.REGION.str.lower()
df_county_FIPS['county_and_state_lower'] = df_county_FIPS.county_and_state.str.lower()

# Helper function for aligning Redfin county names to census county names.
def county_name_matching(county):
  add_city_list = [
      'charlottesville', 'lynchburg', 'norfolk', 'portsmouth', 'harrisonburg',
      'alexandria', 'virginia beach', 'manassas', 'manassas park',
      'chesapeake', 'falls church', 'suffolk', 'hampton', 'fredericksburg',
      'newport news', 'staunton', 'winchester', 'salem', 'petersburg',
      'waynesboro', 'bristol', 'williamsburg', 'hopewell', 'danville',
      'colonial heights', 'poquoson', 'martinsville', 'radford', 'buena vista',
      'lexington', 'covington', 'norton', 'galax', 'emporia'
  ]
  add_city_and_list = [
      'juneau borough', 'sitka borough', 'wrangell borough', 'yakutat borough'
  ]
  drop_county_list = [
      'baltimore city county', 'richmond city county', 'roanoke city county',
      'fairfax city county', 'st. louis city county', 'franklin city county'
  ]
  replace_borough_list = [
      'anchorage borough', 'skagway borough'
  ]
  # If the Redfin county is in the add_city_list, add 'city' to it.
  if county[:-4] in add_city_list:
    return f'{county[:-4]} city{county[-4:]}'
  # If the Redfin county is in the add_city_and_list, add 'city and' to it.
  elif county[:-4] in add_city_and_list:
    return f'{county[:-12]} city and{county[-12:]}'
  # If the Redfin county is in the drop_county_list, drop 'county' from it.
  elif county[:-4] in drop_county_list:
    return f'{county[:-11]}{county[-4:]}'
  # If the Redfin county is in replace_borough_list, change to 'municipality'.
  elif county[:-4] in replace_borough_list:
    return f'{county[:-12]} municipality{county[-4:]}'
  # Replace '&' with 'and' for 'lewis & clark', and 'king & queen' counties.
  elif '&' in county:
    return county.replace('&', 'and')
  # Remove space from 'la salle'.
  elif county=='la salle parish, la':
    return 'lasalle parish, la'
  # If not in the above cases, return original.
  else:
    return county

# Apply the helper function.
df_housing_market['REGION_lower'] = df_housing_market['REGION_lower'].apply(
    county_name_matching
    )
# Add FIPS codes to housing market data, on matching county names.
df_housing_market = pd.merge(
    df_housing_market,
    df_county_FIPS[['county_and_state_lower', 'county_and_state_FIPS']],
    left_on='REGION_lower',
    right_on='county_and_state_lower',
    how='left'
    )
# Two special cases not handled by helper function, add FIPS code directly.
df_housing_market.loc[
    df_housing_market['REGION_lower']==
    'dona ana county, nm', 'county_and_state_FIPS'
    ] = '35013'
df_housing_market.loc[
    df_housing_market['REGION_lower']==
    'valdez-cordova census area, ak', 'county_and_state_FIPS'
    ] = '02261'

In [ ]:
df_housing_market['PERIOD_BEGIN'] = pd.to_datetime(df_housing_market['PERIOD_BEGIN'])
df_housing_market['PERIOD_END'] = pd.to_datetime(df_housing_market['PERIOD_END'])
df_inflation['observation_date'] = pd.to_datetime(df_inflation['observation_date'])


df_inflation = df_inflation.rename(columns={'CPIAUCSL': 'CPIAUCNS'})

df_housing_market = pd.merge(
    df_housing_market,
    df_inflation,
    left_on='PERIOD_BEGIN',
    right_on='observation_date',
    how='left'
)

df_housing_market.drop('observation_date', axis=1, inplace=True)

In [ ]:
df_housing_market = df_housing_market.rename(columns={
    'county_and_state_FIPS':'FIPS_CODE',
    'CPIAUCNS':'CPI'
    })

df_housing_market = df_housing_market[[
    'FIPS_CODE','REGION', 'PROPERTY_TYPE', 'PERIOD_BEGIN',
    'MEDIAN_SALE_PRICE', 'MEDIAN_LIST_PRICE',
    'CPI', 'INVENTORY', 'HOMES_SOLD'
    ]]

In [ ]:
missing_counts = df_housing_market[df_housing_market['MEDIAN_SALE_PRICE'].isna()]['FIPS_CODE'].value_counts().rename('Missing_Count')
total_counts = df_housing_market['FIPS_CODE'].value_counts().rename('Total_Count')
df_missing_sales = pd.merge(
    missing_counts,
    total_counts,
    left_index=True,
    right_index=True,
    how='left'
)

In [ ]:
df_housing_market = df_housing_market.sort_values(by=['FIPS_CODE', 'PROPERTY_TYPE', 'PERIOD_BEGIN'])
df_housing_market = df_housing_market.reset_index().drop(columns=['index'])

In [ ]:
def add_zscore_outlier_cols(df, col):
  # Add columns to track z-score and if a value is an outlier.
  is_outlier_col = col+'_is_outlier'
  zscore_col = col+'_zscore'
  df[zscore_col] = df.groupby(['FIPS_CODE', 'PROPERTY_TYPE'])[[col]].transform(stats.zscore)
  # Define an outlier threshold here.
  max_z = 3
  df[is_outlier_col] = abs(df[zscore_col]>max_z)
  return df

# Add z-score and outlier columns.
price_cols = ['MEDIAN_SALE_PRICE', 'MEDIAN_LIST_PRICE']
for price_col in price_cols:
  df_housing_market = add_zscore_outlier_cols(df_housing_market, price_col)

# Drop z-score columns.
price_cols_zscore = [price_col+'_zscore' for price_col in price_cols]
df_housing_market = df_housing_market.drop(columns=price_cols_zscore)

In [ ]:
def outliers_to_nan(df, col):
  # Add new column that removes values if they are tagged as outliers.
  is_outlier_col = col+'_is_outlier'
  no_outliers_col = col+'_no_outliers'
  df[no_outliers_col] = df[col]
  df.loc[df[is_outlier_col], no_outliers_col] = np.nan
  return df

# Change the price column value to NaN for outliers.
for price_col in price_cols:
  df_housing_market = outliers_to_nan(df_housing_market, price_col)

In [ ]:
# Series for every unique FIPS in the dataset.
all_FIPS = pd.Series(df_housing_market['FIPS_CODE'].unique()).rename('FIPS_CODE')
# Series for every unique property type in the dataset.
all_types = pd.Series(df_housing_market['PROPERTY_TYPE'].unique()).rename('PROPERTY_TYPE')
# Series for every month between min and max values of PERIOD_BEGIN.
all_months = pd.Series(
    pd.date_range(
        start=df_housing_market['PERIOD_BEGIN'].min(),
        end=df_housing_market['PERIOD_BEGIN'].max(),
        freq='MS'# Each month, starting date.
        )
    ).rename('PERIOD_BEGIN')

# Cross join the three series above so every FIPS/property type group
# has rows for every month.
df_all_groups_all_months = pd.merge(
    all_FIPS,
    all_types,
    how='cross'
    ).merge(
        all_months,
        how='cross'
    )

# Before merging our housing market data to many new rows,
# denote current rows as original records.
df_housing_market['original_record'] = True

# Merge our housing market data onto the list of every month for every group.
df_housing_market = pd.merge(
    df_all_groups_all_months,
    df_housing_market,
    on=['FIPS_CODE', 'PROPERTY_TYPE', 'PERIOD_BEGIN'],
    how='left'
    )

# Fill in region and CPI columns.
# For each FIPS, there is only one region value.
df_housing_market['REGION'] =\
  df_housing_market.groupby('FIPS_CODE')['REGION'].ffill()
df_housing_market['REGION'] =\
  df_housing_market.groupby('FIPS_CODE')['REGION'].bfill()
# For each month, theres is only one CPI value.
df_housing_market['CPI'] =\
  df_housing_market.groupby('PERIOD_BEGIN')['CPI'].ffill()
df_housing_market['CPI'] =\
  df_housing_market.groupby('PERIOD_BEGIN')['CPI'].bfill()

# Fill NaN's in "original_record" column, and "_is_outlier" columns with False.
fill_values = {
    'original_record': False,
    'MEDIAN_SALE_PRICE_is_outlier': False,
    'MEDIAN_LIST_PRICE_is_outlier': False
    }
df_housing_market = df_housing_market.fillna(fill_values)

In [ ]:
def interpolate_col(df, col):
  # Create a column to receive interpolated values.
  interpolated_col = col+'_interpolated'
  df[interpolated_col] = np.nan
  # Interpolation will be performed on the column without outliers.
  no_outliers_col = col+'_no_outliers'
  # Create a column to note these values are interpolated.
  is_interpolated_col = col+'_is_interpolated'
  df[is_interpolated_col] = df[no_outliers_col].isna()

  # From: https://stackoverflow.com/questions/37057187/pandas-interpolate-within-a-groupby
  # Define a new df to perform interpolation on.
  df_interpolated = df[['FIPS_CODE', 'PROPERTY_TYPE', 'PERIOD_BEGIN', no_outliers_col]].set_index('PERIOD_BEGIN')
  # Group by FIPS and property type, interpolate for months missing data.
  df_interpolated = df_interpolated.groupby(['FIPS_CODE', 'PROPERTY_TYPE'])\
   [['FIPS_CODE', 'PROPERTY_TYPE', no_outliers_col]].apply(
       lambda group: group.interpolate(
           method='time',
           )
       )
  # Return our original input df, with a new column of interpolated values.
  df[interpolated_col] = df_interpolated[no_outliers_col].values
  return df

# Add interpolated columns.
price_cols = ['MEDIAN_SALE_PRICE', 'MEDIAN_LIST_PRICE']
for col in price_cols:
  df_housing_market = interpolate_col(df_housing_market, col)

# Fill in inventory and homes sold columns.
# For inventory, we assume if there is no reported sales,
# then inventory is constant from last reported sales record.
df_housing_market['INVENTORY'] = \
  df_housing_market.groupby(['FIPS_CODE','PROPERTY_TYPE'])\
  ['INVENTORY'].ffill()
# For homes sold column, we assume if there is no reported sales, 0 homes sold.
df_housing_market = df_housing_market.fillna({'HOMES_SOLD': 0})

# The above fill methods and interpolation will not cover
# groups with earliest months missing (primarily in 2012),
# for these there are limited options, we will backfill.
df_housing_market['MEDIAN_SALE_PRICE_interpolated'] = \
  df_housing_market.groupby(['FIPS_CODE','PROPERTY_TYPE'])\
  ['MEDIAN_SALE_PRICE_interpolated'].bfill()
df_housing_market['MEDIAN_LIST_PRICE_interpolated'] = \
  df_housing_market.groupby(['FIPS_CODE','PROPERTY_TYPE'])\
  ['MEDIAN_LIST_PRICE_interpolated'].bfill()
df_housing_market['INVENTORY'] = \
  df_housing_market.groupby(['FIPS_CODE','PROPERTY_TYPE'])\
  ['INVENTORY'].bfill()

In [ ]:
def inflation_adjustment(df, price_col):
  # Get our earliest record, and the CPI on that date.
  earliest_period = min(df['PERIOD_BEGIN'])
  CPI_init = df[df_housing_market['PERIOD_BEGIN']==earliest_period]['CPI'][0]

  # Create an inflation adjusted column from the price column.
  inflation_col = f'{price_col}_inflation_adj_{str(earliest_period)[0:7]}'
  inflation_adjustment_col = df['CPI']/CPI_init
  df[inflation_col] = df[price_col]*inflation_adjustment_col
  return df

# Add inflation adjusted columns.
price_cols = [
    'MEDIAN_SALE_PRICE', 'MEDIAN_SALE_PRICE_no_outliers',
    'MEDIAN_SALE_PRICE_interpolated',
    'MEDIAN_LIST_PRICE', 'MEDIAN_LIST_PRICE_no_outliers',
    'MEDIAN_LIST_PRICE_interpolated'
    ]
for price_col in price_cols:
  df_housing_market = inflation_adjustment(df_housing_market, price_col)

In [ ]:
df_housing_market_updated = df_housing_market
df_housing_market_updated['Year'] = pd.to_datetime(df_housing_market_updated['PERIOD_BEGIN']).dt.year

In [ ]:
df_housing_market_grouped = df_housing_market_updated.groupby(['FIPS_CODE', 'Year']).agg({
    'MEDIAN_SALE_PRICE': 'median',
    'MEDIAN_LIST_PRICE': 'median'
}).reset_index()

df_housing_market_grouped = df_housing_market_grouped.rename(columns={'FIPS_CODE': 'FIPS'})

df_housing_market_grouped.head(3)

# Calculate Affordability Index

### Initially home affordability index was assumed to be Income-to-Home Value Ratio(median home value / median Income). After further research that was the wrong metric. home affordability index (median income / required income to afford a median priced home). This required estimation in values such at  20% down payment (loan_to_value), mortgage term period, and the gross monthly income that can be used for monthly payments (housing_ratio).

In [ ]:
def calculate_home_affordability_index(income_df, housing_df,
                                       loan_to_value=0.8,
                                       loan_term_years=30,
                                       annual_interest_rate=0.05,
                                       housing_ratio=0.28):
# Ensure Year is integer
    income_df['Year'] = income_df['Year'].astype(int)
    housing_df['Year'] = housing_df['Year'].astype(int)

# Merge datasets
    merged_df = pd.merge(
        income_df[['FIPS', 'Year', 'Median_Household_Income']],
        housing_df[['FIPS', 'Year', 'MEDIAN_SALE_PRICE', 'MEDIAN_LIST_PRICE']],
        on=['FIPS', 'Year'],
        how='left'
    )

# Loan assumptions
    loan_term_months = loan_term_years * 12
    monthly_interest_rate = annual_interest_rate / 12

# Estimate mortgage and affordability
    merged_df['Loan_Amount'] = merged_df['MEDIAN_SALE_PRICE'] * loan_to_value
    merged_df['Monthly_Payment'] = (
        merged_df['Loan_Amount'] * monthly_interest_rate * (1 + monthly_interest_rate) ** loan_term_months
    ) / ((1 + monthly_interest_rate) ** loan_term_months - 1)

    merged_df['Required_Income'] = (merged_df['Monthly_Payment'] * 12) / housing_ratio

    merged_df['HAI'] = np.where(
        merged_df[['Median_Household_Income', 'Required_Income']].notna().all(axis=1),
        (merged_df['Median_Household_Income'] / merged_df['Required_Income']) * 100,
        np.nan
    )

    merged_df_no_na = merged_df.dropna(subset=['HAI'])
    return merged_df_no_na

In [ ]:
merged_df_no_na = calculate_home_affordability_index(
    all_years_data_income,
    df_housing_market_grouped
)


In [ ]:
merged_df_no_na.head()

# Visualization

### Initially this data was visualized using Altair. Their was an issue with the visual not properly displaying the data for multiple regions.I then switched to Plotly and was able to properly visualize the dataset. The altair visual was off due to the data overriding itself when trying to add data slider. If I hardcoded the data, the visual representation was correct but not with the slider. I found plotly to providemore capability with its choropleth visual as well. I also added a trending plot to show the counties some major cities are in to provide additional insight.

In [ ]:
def plot_hai_visualizations(df_input):
# Load US counties GeoJSON
    geojson_url = 'https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json'
    counties_geojson = requests.get(geojson_url).json()

# Prepare data
    df = df_input.copy()
    df['FIPS'] = df['FIPS'].astype(str).str.zfill(5)


# Choropleth Map
    fig_choropleth = px.choropleth(
        df,
        geojson=counties_geojson,
        locations='FIPS',
        color='HAI',
        color_continuous_scale='Plasma',
        range_color=(0, 200),
        scope='usa',
        labels={'HAI': 'Affordability Index'},
        animation_frame='Year',
        title='Home Affordability Index (HAI) by U.S. County Over Time'
    )

    fig_choropleth.update_geos(fitbounds="locations", visible=False)
    fig_choropleth.update_layout(
        margin={"r": 0, "t": 50, "l": 0, "b": 0},
        width=1200,
        height=800
    )
    fig_choropleth.show()

# added line plot visual here with hardcoded FIPS
    city_fips = {
        '36061': 'New York City',
        '12086': 'Miami',
        '06037': 'Los Angeles',
        '17031': 'Chicago'
    }

    df_selected = df[df['FIPS'].isin(city_fips.keys())].copy()
    df_selected['City'] = df_selected['FIPS'].map(city_fips)

    fig_line = px.line(
        df_selected,
        x='Year',
        y='HAI',
        color='City',
        title='Home Affordability Index (HAI) Trend for Selected Cities',
        markers=True
    )

# chose size layout for the visual
    fig_line.update_layout(
        xaxis_title='Year',
        yaxis_title='Home Affordability Index (HAI)',
        width=1000,
        height=600
    )
    fig_line.show()

In [ ]:
plot_hai_visualizations(merged_df_no_na)

# Report Visualization

In [ ]:
fips = pd.read_csv('data/County_FIPS.txt', sep='|')

fips['STATEFP'] = fips['STATEFP'].astype(str).str.zfill(2)
fips['COUNTYFP'] = fips['COUNTYFP'].astype(str).str.zfill(3)
fips['FIPS'] = fips['STATEFP'] + fips['COUNTYFP']

fips['COUNTYSTATE'] = fips['COUNTYNAME'] + ', ' + fips['STATE']

fips.head()

In [ ]:
def plot_hai_visualizations_v2(df_input, fips_codes):
# Load US counties GeoJSON
    geojson_url = 'https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json'
    counties_geojson = requests.get(geojson_url).json()

# Prepare data
    df = df_input.copy()
    df['FIPS'] = df['FIPS'].astype(str).str.zfill(5)

    df = pd.merge(df, fips_codes, on='FIPS', how='inner')

    custom_diverging = [
        "#d95f02",
        "#d3d3d3",
        "#1f77b4"

    ]

    #Fixing the year to export static visuals
    year = 2012
    df_year = df[df['Year'] == year]

    fig_choropleth = px.choropleth(
        df_year,
        geojson=counties_geojson,
        locations='FIPS',
        color='HAI',
        color_continuous_scale=custom_diverging,
        range_color=(0, 200),
        scope='usa',
        hover_data={'COUNTYNAME': True},
        labels={'HAI': 'Affordability Index', 'COUNTYNAME': 'County Name'},
        title=f'Home Affordability Index (HAI) by U.S. County - {year}'
    )

    fig_choropleth.update_geos(
        visible=False,
        projection_type="albers usa"
    )

    fig_choropleth.update_layout(
        margin=dict(l=0, r=0, t=30, b=0),
        width=800,
        height=500,
        title_x=0.5,
        title_y=0.95,
        title_font=dict(size=16),
        coloraxis_colorbar=dict(
            thickness=20,
            len=.7,
            x=0.9,
            y=0.5,
            title="HAI"
        )
    )

    fig_choropleth.show()


# added line plot visual here with hardcoded FIPS
    city_fips = ['06037','17031','48201','04013','12086','36047','36081','06059']

    df_selected = df[df['FIPS'].isin(city_fips)].copy()

    fig_line = px.line(
        df_selected,
        x='Year',
        y='HAI',
        color='COUNTYSTATE',
        title='Home Affordability Index (HAI) Trend for Largest Counties',
        markers=True,
        color_discrete_sequence=px.colors.qualitative.D3
    )

    fig_line.update_layout(
        xaxis_title='Year',
        yaxis_title='Home Affordability Index (HAI)',
        width=1000,
        height=600,
        plot_bgcolor='white',
        paper_bgcolor='white',
        font=dict(size=14),
        legend_title_text='City',
        title_font=dict(size=18)
    )

    fig_line.update_xaxes(showgrid=False)
    fig_line.update_yaxes(showgrid=True, gridcolor='lightgrey')

    fig_line.show()

In [ ]:
plot_hai_visualizations_v2(merged_df_no_na, fips)

In [ ]:
def plot_hai_visualizations_animated_v2(df_input, fips_codes):
# Load US counties GeoJSON
    geojson_url = 'https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json'
    counties_geojson = requests.get(geojson_url).json()

# Prepare data
    df = df_input.copy()
    df['FIPS'] = df['FIPS'].astype(str).str.zfill(5)

    df = pd.merge(df, fips_codes, on='FIPS', how='inner')

    custom_diverging = [
        "#d95f02",
        "#d3d3d3",
        "#1f77b4"

    ]


    fig_choropleth = px.choropleth(
        df,
        geojson=counties_geojson,
        locations='FIPS',
        color='HAI',
        color_continuous_scale=custom_diverging,
        range_color=(0, 200),
        scope='usa',
        hover_data={'COUNTYNAME': True},
        labels={'HAI': 'Affordability Index', 'COUNTYNAME': 'County Name'},
        animation_frame='Year',
        title=f'Home Affordability Index (HAI) by U.S. County'
    )

    fig_choropleth.update_geos(
        visible=False,
        projection_type="albers usa"
    )

    fig_choropleth.update_layout(
        margin=dict(l=0, r=0, t=30, b=0),
        width=800,
        height=500,
        title_x=0.5,
        title_y=0.95,
        title_font=dict(size=16),
        coloraxis_colorbar=dict(
            thickness=20,
            len=.7,
            x=0.9,
            y=0.5,
            title="HAI"
        )
    )

    fig_choropleth.show()

In [ ]:
plot_hai_visualizations_animated_v2(merged_df_no_na, fips)